In [5]:
#使用熵权法进行评估方案
import numpy as np
import pandas as pd

#数据矩阵准备
data=[[1,100,500],[2,200,300],[3,300,200],[4,400,100],[5,500,50]]
data=np.array(data)
data=pd.DataFrame(data)
data

,0,1,2
0,1,100,500
1,2,200,300
2,3,300,200
3,4,400,100
4,5,500,50


In [6]:
#数据矩阵正向化
#将极小型数据转化为极大型数据
def minToMax(maxx,x):
    x=list(x)
    ans=[]
    for i in range(len(x)):
        ans.append(maxx-x[i])
    return ans

#将中间型数据转化为极大型数据
def midToMax(bestx,x):
    x=list(x)
    ans=[]
    h=[]
    #计算列表中每个元素与最优值的差的绝对值
    for i in range (len(x)):
        h.append(abs(bestx-x[i]))
    M=max(h)#计算最大值,用来归一化
    if M==0:
        M=1#防止除0错误
    #计算每个元素的极大型值
    for i in range(len(x)):
        ans.append(1-h[i]/M)
    return np.array(ans)

#将区间型数据转化为极大型数据
def intervalToMax(x,lowx,highx):
    x=list(x)
    ans=[]
    for i in range(len(x)):
        if x[i]>=lowx and x[i]<=highx:
            ans.append(1)
        elif x[i]<lowx:
            ans.append(1-(lowx-x[i])/(lowx-min(x)))
        elif x[i]>highx:
            ans.append(1-(x[i]-highx)/max(x)-highx)
    return np.array(ans)

#对数据矩阵进行标准化
def normalize(data):
    data=np.array(data)
    X=data/np.sqrt(np.sum(data**2,axis=0))
    return X

#对数据矩阵进行归一化
def regularize(data):
    data=np.array(data)
    m,n = np.shape(data)
    for i in range(n):
        col_sum=np.sum(data[:,i])
        for j in range(m):
            data[j,i]=data[j,i]/col_sum
    return data

In [7]:
#对数据矩阵进行正向化
#这里假设第一列是极大型指标，第二列是中间型指标，第三列是极小型指标
data[1]=midToMax(500,data[1])
data[2]=minToMax(500,data[2])
data

,0,1,2
0,1,0.00,0
1,2,0.25,200
2,3,0.50,300
3,4,0.75,400
4,5,1.00,450


In [8]:
#对数据矩阵进行标准化
data=normalize(data)
#对数据矩阵进行归一化得到概率矩阵
data=regularize(data)
data

array([[0.06666667, 0.        , 0.        ],
       [0.13333333, 0.1       , 0.14814815],
       [0.2       , 0.2       , 0.22222222],
       [0.26666667, 0.3       , 0.2962963 ],
       [0.33333333, 0.4       , 0.33333333]])

In [10]:
#计算熵值
def mylog(x):
    n=len(x)#计算向量长度
    lnp=np.zeros(n)#创建一个n个元素的0向量
    for i in range(n):
        if x[i]!=0:
            lnp[i]=np.log(x[i])
        else:
            lnp[i]=0
    return lnp

In [11]:
#定义一个存储熵值的数组
m,n=data.shape
E=np.zeros(n)#n表示的是指标的数目
for j in range(n):
    E[j]=-(1/np.log(m))*np.sum(data[:,j]*mylog(data[:,j]))

E


array([0.92563392, 0.79521814, 0.83492014])

In [12]:
#计算权值
def weight(E):
    n=len(E)
    w=np.zeros(n)
    for j in range(n):
        w[j]=1-E[j]/np.sum(E)
    return w

w=weight(E)
w

array([0.63782613, 0.68885406, 0.67331981])

In [13]:
#根据权值计算各方案得分
def score(data,w):
    m,n=data.shape
    score=np.zeros(m)
    for i in range(m):
        score[i]=np.sum(data[i,:]*w)
    return score

score=score(data,w)
score

array([0.04252174, 0.25367997, 0.41496266, 0.57624535, 0.71259027])

综合下来方案5得分最高，熵权法的优势在于评价指标客观化